In [38]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List
from langchain_community.document_loaders import PyPDFLoader

In [24]:
# get root directory
root_dir = os.getcwd()
# get the path to the data directory
data_dir = os.path.join(root_dir, 'data')

In [26]:

file_name = 'The_Oxford_3000_by_CEFR_Level.pdf'
file_path = os.path.join(data_dir, file_name)

loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [30]:
len(pages)

12

In [40]:

class WordSchema(BaseModel):
    word: str = Field(description="The word")
    pos: str = Field(description="The part of speech")

In [41]:
class OutputSchema(BaseModel):
    words: List[WordSchema] = Field(description="The words extracted from the document")

In [42]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
model_with_structure = model.with_structured_output(OutputSchema)
structured_output = model_with_structure.invoke("extract the word and part of speech from the page: {}".format(pages[0].page_content))

In [46]:
len(structured_output.words)

241

In [47]:
structured_output.words[-1]

WordSchema(word='euro', pos='n.')